In [1]:
import torch
import torch.nn as nn
MAX_GRAD_NORM = 1.2
EPSILON = 50.0
DELTA = 1e-10
EPOCHS = 3

LR = 1e-3

BATCH_SIZE = 120
MAX_PHYSICAL_BATCH_SIZE = 256
from torchvision.models.mobilenet import mobilenet_v2

In [2]:
from torchvision.datasets import MNIST, CIFAR10
import torchvision.transforms as transforms



DATA_ROOT = '../mnist'

transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = CIFAR10(
    root=DATA_ROOT, train=True, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
)

test_dataset = MNIST(
    root=DATA_ROOT, train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
)

print(len(train_dataset))
dataiter = iter(train_loader)
images, labels = next(dataiter)
images[0].size()

Files already downloaded and verified
50000


torch.Size([3, 32, 32])

In [3]:
import numpy as np
import torch.optim as optim
from opacus.utils.batch_memory_manager import BatchMemoryManager


def train(model, train_loader, optimizer, epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []
    
    with BatchMemoryManager(
        data_loader=train_loader, 
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, 
        optimizer=optimizer
    ) as memory_safe_data_loader:

        for i, (images, target) in enumerate(memory_safe_data_loader):   
            optimizer.zero_grad()
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            # measure accuracy and record loss
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

            loss.backward()
            optimizer.step()
            

            if (i+1) % 200 == 0:
                epsilon = privacy_engine.get_epsilon(DELTA)
                print(
                    f"\tTrain Epoch: {epoch} \t"
                    f"Loss: {np.mean(losses):.6f} "
                    f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
                    f"(ε = {epsilon:.5f}, δ = {DELTA})"
                )
    return np.mean(top1_acc), epsilon
    

In [4]:
def test(model, test_loader, device):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    losses = []
    top1_acc = []

    with torch.no_grad():
        for images, target in test_loader:
            images = images.to(device)
            target = target.to(device)

            output = model(images)
            loss = criterion(output, target)
            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()
            acc = accuracy(preds, labels)

            losses.append(loss.item())
            top1_acc.append(acc)

    top1_avg = np.mean(top1_acc)

    print(
        f"\tTest set:"
        f"Loss: {np.mean(losses):.6f} "
        f"Acc: {top1_avg * 100:.6f} "
    )
    return np.mean(top1_acc)


from tqdm.notebook import tqdm

report = []

def accuracy(preds, labels):
    return (preds == labels).mean()

#for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
#    results = train(model, train_loader, optimizer, epoch + 1, device)
#    report.append(results)

In [5]:
from opacus import PrivacyEngine
import torch
from torch import nn
from torchvision import models
from opacus.validators import ModuleValidator
model = models.resnet18(num_classes=10)

In [6]:
import pandas as pd
G_h =['eps_check','distortion','clip','q','k','theta']
GU_h =['eps_check','distortion','clip','q', 'a', 'b', 'k','theta', 'M']
#G250 = pd.read_csv("../arguments/G250_values.csv", names = G_h)
#GU250 = pd.read_csv("../arguments/GU_10Evalues.csv", names=GU_h)
#N250 = pd.read_csv("../arguments/N250_values.csv")

In [7]:
EPSILON = 5

GU_h =['eps_check','distortion','clip','q', 'a', 'b', 'k','theta', 'M']
GU250 = pd.read_csv("../arguments/GU_values.csv", names=GU_h)
GU250 = GU250.sort_values("eps_check")
GU250.replace([np.inf, -np.inf], np.nan, inplace=True)
GU250 = GU250.dropna()
GU250 = GU250.reset_index()
GU250.loc[(GU250['eps_check']  <= EPSILON) & (GU250['eps_check']  > EPSILON-0.1)].sort_values("distortion")

,index,eps_check,distortion,clip,q,a,b,k,theta,M
65546,8810,4.956052,0.000635,0.1,0.0024,0.000000,0.000010,37.626263,0.000625,1234.476227
65550,18158,4.957413,0.000638,0.1,0.0024,0.000010,0.000013,37.626263,0.000625,1234.816610
65547,27506,4.956372,0.000638,0.1,0.0024,0.000000,0.000013,37.626263,0.000625,1234.556202
65548,55550,4.956790,0.000642,0.1,0.0024,0.000000,0.000017,37.626263,0.000625,1234.660738
65551,36854,4.957832,0.000642,0.1,0.0024,0.000010,0.000017,37.626263,0.000625,1234.921142
...,...,...,...,...,...,...,...,...,...,...
65498,346806,4.904135,0.014843,0.1,0.0024,0.000013,0.000085,1.495360,0.014758,1436.799387
65499,356154,4.904553,0.014843,0.1,0.0024,0.000017,0.000085,1.495360,0.014758,1436.903886
65501,365502,4.905100,0.014843,0.1,0.0024,0.000022,0.000085,1.495360,0.014758,1437.040479
65504,384198,4.906748,0.014843,0.1,0.0024,0.000038,0.000085,1.495360,0.014758,1437.452402


In [8]:
GU250['eps_check'][69313]

6.89541495212608

In [9]:
GU250.loc[(GU250['eps_check']  <= 0.5) & (GU250['eps_check']  > 0.49)].sort_values("distortion")

,index,eps_check,distortion,clip,q,a,b,k,theta,M
38231,9058,0.493228,0.000065,0.1,0.0024,0.000000,0.000010,41.464646,0.000055,118.869099
38261,18406,0.494589,0.000068,0.1,0.0024,0.000010,0.000013,41.464646,0.000055,119.209482
38238,27754,0.493548,0.000068,0.1,0.0024,0.000000,0.000013,41.464646,0.000055,118.949074
38268,37102,0.495007,0.000072,0.1,0.0024,0.000010,0.000017,41.464646,0.000055,119.314014
38247,55798,0.493966,0.000072,0.1,0.0024,0.000000,0.000017,41.464646,0.000055,119.053611
...,...,...,...,...,...,...,...,...,...,...
38346,375140,0.499103,0.001381,0.1,0.0024,0.000029,0.000085,1.709987,0.001296,120.337926
38314,347096,0.497424,0.001381,0.1,0.0024,0.000013,0.000085,1.709987,0.001296,119.918291
38320,356444,0.497842,0.001381,0.1,0.0024,0.000017,0.000085,1.709987,0.001296,120.022791
38307,337748,0.497105,0.001381,0.1,0.0024,0.000010,0.000085,1.709987,0.001296,119.838345


In [10]:
acc_plrv_2 = []
from torchvision.models.mobilenet import mobilenet_v2
from transformers import ViTForImageClassification, ViTConfig
for i in [1]:
    torch.cuda.empty_cache()
    args ={
        "a1":1,
        "a3":1,
        "a4":1,
        "lam":1,
        "moment":1,
        "theta":0.00001,
        'k':32.6884422110553,
        'mu':0,
        'sigma':0.5,
        'a':GU250['a'][i],
        'b':GU250['b'][i],
        'u':1,
        'l':0.1,
        'epsilon':1,
        'max_grad_norm': 2,
        'gamma':True,
        'uniform':False,
        'truncnorm':False,
    }
    print(GU250['theta'][i])
    EPOCHS = 5
    BATCH_SIZE = 119
    #print()
    from torchvision.datasets import MNIST
    import torchvision.transforms as transforms



    DATA_ROOT = '../mnist'

    transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    train_dataset = CIFAR10(
        root=DATA_ROOT, train=True, download=True, transform=transform)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
    )

    test_dataset = CIFAR10(
        root=DATA_ROOT, train=False, download=True, transform=transform)

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
    )
    
    #gen_args = find_values(15, i)
    runs = []
    trun = []

    model = mobilenet_v2(num_classes=10)
    #model.classifier[1] = torch.nn.Linear(in_features=model.classifier[1].in_features, out_features=10)
    #model.conv1 = nn.Conv2d(1, 28, kernel_size=3, stride=1, padding=1, bias=False)
    #model.load_state_dict(dic)
    #model.train()


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ModuleValidator.fix(model)
    model = model.to(device)

    privacy_engine = PrivacyEngine(accountant = 'rdp_plrv')

    optimizer = optim.RMSprop(model.parameters(), lr=LR)
    print("Begin training " + str(i))
    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
            module=model,
            optimizer=optimizer,
            data_loader=train_loader,
            noise_multiplier = 1,
            epochs=EPOCHS,
            target_epsilon=3,
            target_delta=DELTA,
            max_grad_norm=GU250['clip'][i],
            PLRV_args=args,
    )

    plrv_report_acc = []
    plrv_report_ep = []

    for epoch in tqdm(range(EPOCHS), desc="Epoch", unit="epoch"):
        acc, ep = train(model, train_loader, optimizer, epoch + 1, device)
        plrv_report_acc.append(acc)
        plrv_report_ep.append(ep)

    acc_plrv_2.append(plrv_report_ep)
    del model
    del optimizer
    #del results
#    torch.cuda.empty_cache()

1e-05
Files already downloaded and verified
Files already downloaded and verified
Begin training 1


/home/knil/Documents/NextCloud/RAstuff/opacus/opacus/privacy_engine.py:100: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?epoch/s]

/home/knil/.local/lib/python3.12/site-packages/torch/nn/modules/module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/home/knil/Documents/NextCloud/RAstuff/opacus/opacus/accountants/analysis/rdp_plrv.py:106: RuntimeWarning: invalid value encountered in scalar divide
  return n/d
/home/knil/Documents/NextCloud/RAstuff/opacus/opacus/accountants/analysis/rdp_plrv.py:204: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


200
	Train Epoch: 1 	Loss: 2.497953 Acc@1: 10.235172 (ε = 0.07017, δ = 1e-10)
400
	Train Epoch: 1 	Loss: 2.405777 Acc@1: 10.640255 (ε = 0.07406, δ = 1e-10)
621
	Train Epoch: 2 	Loss: 2.148291 Acc@1: 16.253793 (ε = 0.07835, δ = 1e-10)
821
	Train Epoch: 2 	Loss: 2.101546 Acc@1: 17.786332 (ε = 0.08223, δ = 1e-10)
1042
	Train Epoch: 3 	Loss: 2.135186 Acc@1: 25.924064 (ε = 0.08652, δ = 1e-10)
1242
	Train Epoch: 3 	Loss: 2.326306 Acc@1: 28.552486 (ε = 0.09040, δ = 1e-10)
1463
	Train Epoch: 4 	Loss: 2.539620 Acc@1: 34.999095 (ε = 0.09469, δ = 1e-10)
1663
	Train Epoch: 4 	Loss: 2.635543 Acc@1: 37.543101 (ε = 0.09857, δ = 1e-10)
1884
	Train Epoch: 5 	Loss: 2.796383 Acc@1: 44.069685 (ε = 0.10286, δ = 1e-10)
2084
	Train Epoch: 5 	Loss: 2.771918 Acc@1: 45.152382 (ε = 0.10674, δ = 1e-10)


In [11]:
acc_plrv_2 = []
from torchvision.models.mobilenet import mobilenet_v2
for i in [65546, 38231]:
    torch.cuda.empty_cache()
    args ={
        "a1":1,
        "a3":1,
        "a4":1,
        "lam":1,
        "moment":1,
        "theta":GU250['theta'][i],
        'k':GU250['k'][i],
        'mu':0,
        'sigma':0.5,
        'a':GU250['a'][i],
        'b':GU250['b'][i],
        'u':1,
        'l':0.1,
        'epsilon':1,
        'max_grad_norm': GU250['clip'][i],
        'gamma':True,
        'uniform':True,
        'truncnorm':False,
    }
    print(GU250['theta'][i])
    EPOCHS = 5
    BATCH_SIZE = 119
    #print()
    from torchvision.datasets import MNIST
    import torchvision.transforms as transforms



    DATA_ROOT = '../mnist'

    transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    train_dataset = CIFAR10(
        root=DATA_ROOT, train=True, download=True, transform=transform)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
    )

    test_dataset = CIFAR10(
        root=DATA_ROOT, train=False, download=True, transform=transform)

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
    )
    
    #gen_args = find_values(15, i)
    runs = []
    trun = []

    model = mobilenet_v2(num_classes=10)
    #model.classifier[1] = torch.nn.Linear(in_features=model.classifier[1].in_features, out_features=10)
    #model.conv1 = nn.Conv2d(1, 28, kernel_size=3, stride=1, padding=1, bias=False)
    #model.load_state_dict(dic)
    #model.train()


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ModuleValidator.fix(model)
    model = model.to(device)

    privacy_engine = PrivacyEngine(accountant = 'rdp_plrv')

    optimizer = optim.RMSprop(model.parameters(), lr=LR)
    print("Begin training " + str(i))
    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
            module=model,
            optimizer=optimizer,
            data_loader=train_loader,
            noise_multiplier = 1,
            epochs=EPOCHS,
            target_epsilon=3,
            target_delta=DELTA,
            max_grad_norm=GU250['clip'][i],
            PLRV_args=args,
    )

    plrv_report_acc = []
    plrv_report_ep = []
    privacy_engine.accountant.sample_rate = 1/len(train_loader)
    epsis = []
    for j in range(len(train_loader)*5):
        privacy_engine.accountant.history = [[args, j]]
        epsis.append(privacy_engine.get_epsilon(10e-10))
    acc_plrv_2.append(epsis)

0.000624865811116
Files already downloaded and verified
Files already downloaded and verified
Begin training 65546
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248

1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038


1571
1572
1573
1574
1575
1576
1577
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770


In [12]:
acc_plrv_2 = [[(10.827088,0.96580), 
              (27.407932,1.96362),
              (41.499464,2.96142),
              (47.529392,3.95914),
              (52.126053,4.95723),
              ], [(10.192589,0.10905),
              (16.807797,0.20513),
              (27.030008,0.30121),
              (38.087848,0.39730),
              (46.760616,0.49338),]
             ]

In [13]:
max(acc_plrv_2[0])

(52.126053, 4.95723)

In [14]:
acc_plrv = []
for i in range(1, len(GU250), int(len(GU250)/10)):
    torch.cuda.empty_cache()
    args ={
        "a1":1,
        "a3":1,
        "a4":1,
        "lam":1,
        "moment":1,
        "theta":GU250['theta'][i],
        'k':GU250['k'][i],
        'mu':0,
        'sigma':0.5,
        'a':GU250['a'][i],
        'b':GU250['b'][i],
        'u':1,
        'l':0.1,
        'epsilon':1,
        'max_grad_norm': GU250['b'][i],
        'gamma':True,
        'uniform':False,
        'truncnorm':False,
    }
    
    #gen_args = find_values(15, i)
    runs = []
    trun = []

    model = mobilenet_v2(num_classes=10)
    #model.classifier[1] = torch.nn.Linear(in_features=model.classifier[1].in_features, out_features=10)
    #model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    #model.load_state_dict(dic)
    #model.train()


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ModuleValidator.fix(model)
    model = model.to(device)

    privacy_engine = PrivacyEngine(accountant = 'rdp_plrv')

    optimizer = optim.RMSprop(model.parameters(), lr=LR)
    print("Begin training " + str(i))
    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
            module=model,
            optimizer=optimizer,
            data_loader=train_loader,
            noise_multiplier = 1,
            epochs=10,
            target_epsilon=0.15,
            target_delta=DELTA,
            max_grad_norm=GU250['clip'][i],
            PLRV_args=args,
    )

    plrv_report_acc = []
    plrv_report_ep = []

    for epoch in tqdm(range(5), desc="Epoch", unit="epoch"):
        acc, ep = train(model, train_loader, optimizer, epoch + 1, device)
        plrv_report_acc.append(acc)
        plrv_report_ep.append(ep)

    acc_plrv.append((G250['distortion'][i], test(model, test_loader, device)))
    del model
    del optimizer
    #del results
#    torch.cuda.empty_cache()

Begin training 1


Epoch:   0%|          | 0/5 [00:00<?, ?epoch/s]

200
	Train Epoch: 1 	Loss: 2.463116 Acc@1: 9.921548 (ε = 0.06642, δ = 1e-10)
400
	Train Epoch: 1 	Loss: 2.392245 Acc@1: 10.218374 (ε = 0.06654, δ = 1e-10)
621
	Train Epoch: 2 	Loss: 2.090827 Acc@1: 21.345325 (ε = 0.06669, δ = 1e-10)
821
	Train Epoch: 2 	Loss: 2.124887 Acc@1: 26.013381 (ε = 0.06681, δ = 1e-10)
1042
	Train Epoch: 3 	Loss: 2.299551 Acc@1: 34.715880 (ε = 0.06695, δ = 1e-10)
1242
	Train Epoch: 3 	Loss: 2.414221 Acc@1: 35.714143 (ε = 0.06708, δ = 1e-10)
1463
	Train Epoch: 4 	Loss: 2.553982 Acc@1: 39.605806 (ε = 0.06722, δ = 1e-10)
1663
	Train Epoch: 4 	Loss: 2.609270 Acc@1: 40.977427 (ε = 0.06735, δ = 1e-10)
1884
	Train Epoch: 5 	Loss: 2.679561 Acc@1: 44.054921 (ε = 0.06749, δ = 1e-10)
2084
	Train Epoch: 5 	Loss: 2.665736 Acc@1: 44.984952 (ε = 0.06761, δ = 1e-10)


NameError: name 'G250' is not defined

In [ ]:
acc_plrv

In [ ]:
g_eps = pd.read_csv('../arguments/gaussian_eps.csv', names = ['eps'])
g_dist = pd.read_csv('../arguments/gaussian_dist.csv', names = ['distortion'])
g_clip = pd.read_csv('../arguments/gaussian_clip.csv', names = ['clip'])

In [ ]:
g_clip = g_clip.dropna()
g_clip = g_clip.reset_index()

In [ ]:
acc_rdp = []
from torchvision.models.mobilenet import mobilenet_v2
for i in [0.5, 5]:
    print(i)
    #gen_args = find_values(15, i)
    runs = []
    trun = []

    model = mobilenet_v2(num_classes=10)
    #model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    #model.load_state_dict(dic)
    #model.train()


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ModuleValidator.fix(model)
    model = model.to(device)
    
    privacy_engine = PrivacyEngine(accountant = 'rdp')

    optimizer = optim.RMSprop(model.parameters(), lr=LR)
    print("Begin training " + str(i))
    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
            module=model,
            optimizer=optimizer,
            data_loader=train_loader,
            #noise_multiplier = 1,
            epochs=5,
            target_epsilon=i,
            target_delta=1e-10,
            max_grad_norm=10,
            #PLRV_args=convert_params(gen_args),
    )

    plrv_report_acc = []
    plrv_report_ep = []

    for epoch in tqdm(range(5), desc="Epoch", unit="epoch"):
        acc, ep = train(model, train_loader, optimizer, epoch + 1, device)
        plrv_report_acc.append(acc)
        plrv_report_ep.append(ep)

    acc_rdp.append((plrv_report_acc, plrv_report_ep))
    del model
    del optimizer
    #del results
    torch.cuda.empty_cache()
    
    privacy_engine.accountant.sample_rate = 1/len(train_loader)
    epsis = []
    for i in range(len(train_loader)*5):
        privacy_engine.accountant.history = [[args, i]]
        epsis.append(privacy_engine.get_epsilon(10e-10))
    acc_rdp.append(epsis)

In [ ]:
from opacus.accountants.utils import get_noise_multiplier
acc_rdp = []
from torchvision.models.mobilenet import mobilenet_v2
for i in [0.5, 5]:
    print(i)
    #gen_args = find_values(15, i)
    runs = []
    trun = []

    model = mobilenet_v2(num_classes=10)
    #model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    #model.load_state_dict(dic)
    #model.train()


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ModuleValidator.fix(model)
    model = model.to(device)
    
    privacy_engine = PrivacyEngine(accountant = 'rdp')

    optimizer = optim.RMSprop(model.parameters(), lr=LR)
    print("Begin training " + str(i))
    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
            module=model,
            optimizer=optimizer,
            data_loader=train_loader,
            #noise_multiplier = 1,
            epochs=5,
            target_epsilon=i,
            target_delta=1e-10,
            max_grad_norm=10,
            #PLRV_args=convert_params(gen_args),
    )

    plrv_report_acc = []
    plrv_report_ep = []
    
    nm = noise_multiplier=get_noise_multiplier(
                target_epsilon=i,
                target_delta=DELTA,
                sample_rate=1/len(train_loader),
                epochs=EPOCHS,
                accountant=privacy_engine.accountant.mechanism(),
            )
    print(nm)
    #privacy_engine.accountant.sample_rate = BATCH_SIZE/50000
    epsis = []
    for j in range(len(train_loader)*5):
        privacy_engine.accountant.history = [(nm, 1/len(train_loader), j)]
        epsis.append(privacy_engine.get_epsilon(DELTA))
    acc_rdp.append(epsis)

In [ ]:
 acc_rdp

In [ ]:
acc_rdp = [[(10.116650,0.43046), 
              (10.711590,0.44728),
              (10.492322,0.46410),
              (10.288952,0.48092),
              (10.566023,0.49774),
              ], [(10.417261,4.40004),
              (10.726745,4.60714),
              (11.139498,4.75362),
              (12.236695,4.87810),
              (12.436983,4.99017),]
             ]

In [ ]:

model = models.resnet18(num_classes=10)
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model = ModuleValidator.fix(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
privacy_engine = PrivacyEngine(accountant = 'rdp_plrv')

optimizer = optim.RMSprop(model.parameters(), lr=LR)
model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
            module=model,
            optimizer=optimizer,
            data_loader=train_loader,
            noise_multiplier = 1,
            epochs=10,
            target_epsilon=0.15,
            target_delta=DELTA,
            max_grad_norm=GU250['clip'][i],
            PLRV_args=args,
    )
dic = model.state_dict()

In [ ]:
del model

In [ ]:
dist_x = [i[0] for i in acc_plrv_2]
acc_y = [i[1] for i in acc_plrv_2]
dist_g = [i[0] for i in acc_rdp]
acc_g = [i[1] for i in acc_rdp]
import matplotlib.pyplot as plt
plt.xlabel("distortion")
plt.ylabel("Accuracy")
plt.title("distortion vs accuracy")
plt.yscale('log')
fig, ax1 = plt.subplots()
ax1.plot(dist_x, acc_y, 'bo-', label="PLRV")
ax1.set_ylabel("Accuracy")
#ax1.set_yscale('log')
ax2 = ax1.twiny()
ax2.plot(dist_g, acc_g, 'ro-', label="Guassian")
#ax2.set_yscale('log')
plt.legend(loc="lower right")
plt.show()

In [ ]:
acc_plrv_2.sort()
acc_rdp.sort()
dist_x = [i[2] for i in acc_plrv_2]
acc_y = [i[1] for i in acc_plrv_2]
dist_g = [i[0] for i in acc_rdp]
acc_g = [i[1] for i in acc_rdp]
import matplotlib.pyplot as plt
plt.xlabel("distortion")
plt.ylabel("Accuracy")
plt.title("distortion vs accuracy")
plt.yscale('log')
fig, ax1 = plt.subplots()
ax1.plot(dist_x, acc_y, 'bo-', label="PLRV")
ax1.set_ylabel("Accuracy")
ax1.set_xscale('log')
ax2 = ax1.twiny()
ax2.plot(dist_g, acc_g, 'ro-', label="Guassian")
#ax2.set_yscale('log')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import matplotlib.ticker as ticker
#acc_plrv_2.sort()
#acc_rdp.sort()
x_vals = range(1,6)
print(acc_plrv_2)
plrv1 = [i[1] for i in acc_plrv_2[0]]
plrv2 = [i[1] for i in acc_plrv_2[1]]
print(acc_rdp)
g1 = [i[1] for i in acc_rdp[0]]
g2 = [i[1] for i in acc_rdp[1]]
import matplotlib.pyplot as plt
plt.xlabel("Epochs")
plt.ylabel("Epsilon")
plt.title("Epochs vs Epsilon")
#plt.plot(x_vals, plrv1, 'b+-', label="PLRV 45% accuracy")
#plt.plot(x_vals, plrv2, 'bo--', label="PLRV 10% accuracy")
plt.plot(x_vals, g1, 'ro-', label="gaussian ϵ=0.5")
#plt.plot(x_vals, g2, 'r+-', label="gaussian ϵ=5")
plt.plot(x_vals, plrv2, 'bo-', label="PLRV  ϵ=0.5")
#plt.plot(x_vals, plrv1, 'b+-', label="PLRV  ϵ=5")
#plt.yticks(np.logspace(0, 200, 5)) 
#plt.yscale("log")
#plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x:.2f}'))
plt.legend(loc="best")
plt.show()

In [ ]:
import matplotlib.ticker as ticker
#acc_plrv_2.sort()
#acc_rdp.sort()
x_vals = range(1,2106)

plrv1 = acc_plrv_2[0]
plrv2 = acc_plrv_2[1]
g2 = acc_rdp[1]
g1 = acc_rdp[0]
import matplotlib.pyplot as plt
plt.xlabel("Steps")
plt.ylabel("Epsilon")
plt.title("Steps vs Epsilon")
#plt.plot(x_vals, plrv1, 'b+-', label="PLRV 45% accuracy")
#plt.plot(x_vals, plrv2, 'bo--', label="PLRV 10% accuracy")
plt.plot(x_vals, g1, 'r--', label="gaussian ϵ=0.5")
plt.plot(x_vals, g2, 'r-', label="gaussian ϵ=5")
plt.plot(x_vals, plrv2, 'b--', label="PLRV  ϵ=0.5")
plt.plot(x_vals, plrv1, 'b-', label="PLRV  ϵ=5")
#plt.yticks(np.logspace(0, 200, 5)) 
#plt.yscale("log")
#plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x:.2f}'))
plt.legend(loc="best")
plt.show()

In [ ]:
import matplotlib.ticker as ticker
#acc_plrv_2.sort()
#acc_rdp.sort()
x_vals = range(1,len(train_loader)+1)

plrv1 = acc_plrv_2[0][0:len(train_loader)]
plrv2 = acc_plrv_2[1][0:len(train_loader)]
g2 = acc_rdp[1][0:len(train_loader)]
g1 = acc_rdp[0][0:len(train_loader)]
import matplotlib.pyplot as plt
plt.xlabel("Steps")
plt.ylabel("Epsilon")
plt.title("Steps vs Epsilon")
#plt.plot(x_vals, plrv1, 'b+-', label="PLRV 45% accuracy")
#plt.plot(x_vals, plrv2, 'bo--', label="PLRV 10% accuracy")
plt.plot(x_vals, g1, 'r--', label="gaussian ϵ=0.5")
plt.plot(x_vals, g2, 'r-', label="gaussian ϵ=5")
plt.plot(x_vals, plrv2, 'b--', label="PLRV  ϵ=0.5")
plt.plot(x_vals, plrv1, 'b-', label="PLRV  ϵ=5")
#plt.yticks(np.logspace(0, 200, 5)) 
#plt.yscale("log")
#plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x:.2f}'))
plt.legend(loc="best")
plt.show()

In [ ]:
import matplotlib.ticker as ticker
#acc_plrv_2.sort()
#acc_rdp.sort()
x_vals = range(1,len(train_loader)+1)

plrv1 = acc_plrv_2[0][0:len(train_loader)]
plrv2 = acc_plrv_2[1][0:len(train_loader)]
g2 = acc_rdp[1][0:len(train_loader)]
g1 = acc_rdp[0][0:len(train_loader)]
import matplotlib.pyplot as plt
plt.xlabel("Steps")
plt.ylabel("Epsilon")
plt.title("Steps vs Epsilon")
#plt.plot(x_vals, plrv1, 'b+-', label="PLRV 45% accuracy")
#plt.plot(x_vals, plrv2, 'bo--', label="PLRV 10% accuracy")
plt.plot(x_vals, g1, 'r--', label="gaussian ϵ=0.5")
#plt.plot(x_vals, g2, 'r-', label="gaussian ϵ=5")
plt.plot(x_vals, plrv2, 'b--', label="PLRV  ϵ=0.5")
#plt.plot(x_vals, plrv1, 'b-', label="PLRV  ϵ=5")
#plt.yticks(np.logspace(0, 200, 5)) 
#plt.yscale("log")
#plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, _: f'{x:.2f}'))
plt.legend(loc="best")
plt.show()

In [ ]:
acc_plrv = []
for i in range(1, 11):
    gen_args = find_values(13, i)
    runs = []
    trun = []

    #model = models.resnet18(num_classes=10)
    #model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    model.load_state_dict(dic)
    model.train()


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)

    privacy_engine = PrivacyEngine(accountant = 'rdp_plrv')

    optimizer = optim.RMSprop(model.parameters(), lr=LR)
    print("Begin training " + str(i))
    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
            module=model,
            optimizer=optimizer,
            data_loader=train_loader,
            noise_multiplier = 1,
            epochs=EPOCHS,
            target_epsilon=0.15,
            target_delta=DELTA,
            max_grad_norm=i+1,
            PLRV_args=convert_params(gen_args),
    )

    plrv_report_acc = []
    plrv_report_ep = []

    for epoch in tqdm(range(1), desc="Epoch", unit="epoch"):
        acc, ep = train(model, train_loader, optimizer, epoch + 1, device)
        plrv_report_acc.append(acc)
        plrv_report_ep.append(ep)

    acc_plrv.append(test(model, test_loader, device))
#    del model
    del optimizer
    #del results
#    torch.cuda.empty_cache()

In [ ]:
3**False

In [ ]:
import sklearn.linear_model as lm

lm.LinearRegression().fit(X, plrv1)

In [ ]:
X = [50000/120*1,50000/120*2,50000/120*3,50000/120*4,50000/120*5]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to calculate the slope and intercept of the linear regression line
def linear_regression(x, y):
    # Using the formula for linear regression: y = mx + b
    # m = (N * Σ(xy) - Σx * Σy) / (N * Σ(x^2) - (Σx)^2)
    # b = (Σy - m * Σx) / N

    N = len(x)
    Σx = np.sum(x)
    Σy = np.sum(y)
    Σxy = np.sum(x * y)
    Σx2 = np.sum(x ** 2)

    # Calculate slope (m) and intercept (b)
    m = (N * Σxy - Σx * Σy) / (N * Σx2 - Σx ** 2)
    b = (Σy - m * Σx) / N

    return m, b

# Function to predict y values based on the linear model
def predict(x, m, b):
    return m * x + b

# Example input data (x and y)
x = np.array(X)
y = np.array(plrv1)

# Calculate the slope and intercept
m, b = linear_regression(x, y)

# Display the result
print(f"Linear regression equation: y = {m:.2f}x + {b:.2f}")

# Predict y values based on the regression line
y_pred = predict(x, m, b)

# Plot the data points and the regression line
plt.scatter(x, y, color='blue')
plt.plot(x, y_pred, color='red', label=f'Regression line: y = {m:.2E}x + {b:.2f}')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('PLRV Linear Regression')
plt.legend()
plt.show()

In [ ]:
x = np.array(X)
y = np.array(plrv2)

# Calculate the slope and intercept
m, b = linear_regression(x, y)

# Display the result
print(f"Linear regression equation: y = {m:.2f}x + {b:.2f}")

# Predict y values based on the regression line
y_pred = predict(x, m, b)

# Plot the data points and the regression line
plt.scatter(x, y, color='blue', label='Data points')
plt.plot(x, y_pred, color='red', label=f'Regression line: y = {m:.2E}x + {b:.2f}')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Linear Regression')
plt.legend()
plt.show()

In [ ]:
x = np.array(X)
y = np.log(np.array(g1))

# Calculate the slope and intercept
m, b = linear_regression(x, y)

# Display the result
print(f"Linear regression equation: y = {m:.2E}x + {b:.2f}")

# Predict y values based on the regression line
y_pred = predict(x, m, b)

# Plot the data points and the regression line
plt.scatter(x, y, color='blue', label='Data points')
plt.plot(x, y_pred, color='red', label=f'Regression line: y = {m:.2f}ln(x) + {b:.2f}')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Linear Regression')
plt.legend()
plt.show()

In [ ]:
x = np.log(np.array(X))
y = np.array(g2)

# Calculate the slope and intercept
m, b = linear_regression(x, y)

# Display the result
print(f"Linear regression equation: y = {m:.2E}ln(x) + {b:.2f}")

# Predict y values based on the regression line
y_pred = predict(x, m, b)

# Plot the data points and the regression line
plt.scatter(x, y, color='blue', label='Data points')
plt.plot(x, y_pred, color='red', label=f'Regression line: y = {m:.2f}ln(x) + {b:.2f}')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Linear Regression')
plt.legend()
plt.show()

In [ ]:
len(acc_plrv_2[1])

In [ ]:
import torch
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import torch
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

# Load the Question NLI dataset
dataset = load_dataset("snli")
# Load the Roberta tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Preprocess the data
def preprocess_function(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True, padding=True)

# Apply preprocessing to train, validation, and test sets
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets = tokenized_datasets.filter(lambda example: example['labels'] != -1)

# Load the Roberta model for sequence classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

# Define the metric for evaluation
def compute_metrics(p):
    predictions, labels = p
    predictions = torch.argmax(torch.tensor(predictions), dim=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",# Evaluate after every epoch
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    logging_dir="./logs",
    load_best_model_at_end=True,
)

device = torch.device("cpu")
model.to(device)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=lambda data: tokenizer.pad(data, padding=True, return_tensors="pt")  # Use dynamic padding
)

trainer.args.device = "cpu"

# Train the model
trainer.train()

# Evaluate on the test set
test_results = trainer.evaluate(tokenized_datasets["test"])

# Print the final test results
print(f"Test results: {test_results}")


In [ ]:
print(set(tokenized_datasets['train']['labels']))  # Check label values in the training set
print(set(tokenized_datasets['validation']['labels']))  # Check label values in the validation set

In [ ]:
max(tokenized_datasets['train']['labels'])

In [ ]:
import os
from datasets import load_dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
import torch

# Ensure that the model runs on the CPU
device = torch.device("cpu")

# Load the Question NLI dataset
dataset = load_dataset("snli")

# Load the Roberta tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Preprocess the data
def preprocess_function(examples):
    # Tokenize the premise and hypothesis
    return tokenizer(
        examples['premise'], 
        examples['hypothesis'], 
        truncation=True, 
        padding='max_length', 
        max_length=512  # Ensure input size consistency
    )

# Apply preprocessing to train, validation, and test sets
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Ensure that the labels are correctly passed as "labels" in the dataset
# In SNLI, the labels are already present in the 'label' field. We rename it to 'labels'
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Remove samples with label -1 (if they exist)
tokenized_datasets = tokenized_datasets.filter(lambda example: example['labels'] != -1)

# Check to ensure there are no remaining -1 labels
print(set(tokenized_datasets['train']['labels']))  # Check the label values after filtering

# Load the Roberta model for sequence classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

# Move the model to CPU
#model.to(device)

# Define the metric for evaluation
def compute_metrics(p):
    predictions, labels = p
    predictions = torch.argmax(torch.tensor(predictions), dim=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Set up training arguments to run on CPU
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate after every epoch
    save_strategy="epoch",        # Save after every epoch
    per_device_train_batch_size=8,  # Lower batch size for CPU
    per_device_eval_batch_size=8,   # Lower batch size for CPU
    num_train_epochs=3,
    logging_dir="./logs",
    load_best_model_at_end=True,
    use_cpu=False,  # Disable CUDA to force CPU
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate on the test set
test_results = trainer.evaluate(tokenized_datasets["test"])

# Print the final test results
print(f"Test results: {test_results}")


In [ ]:
!pip install datasets transformers[torch] scikit-learn